In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Math
from matplotlib.font_manager import FontProperties
from matplotlib import rc
import matplotlib as mpl
import sys
sys.path.append("..")
from exotop import topo_1D_plots as plottop
from exotop.model_1D import inputs as ins
from exotop.model_1D import parameters as p

# set paths
# fig_path = '/space/cmg76/figs/' # office
fig_path = 'figs/' # laptop#
fig_format = '.png'
benchmark_path = '../benchmarks/'

# set mpl display parameters
mpl.rcParams['axes.linewidth'] = 2  # set the value globally - for poster
rc('font',**{'family':'serif','serif':['Times New Roman']})  # latex font for matplotlib CMU Serif Roman
# rc('font',**{'family':'monospace','monospace':['Ubuntu Mono']})  # for computery poster 
rc('text', usetex=True)  # turn on for latexy font
mpl.font_manager._rebuild()
plt.style.use('dark_background')

In [73]:
##### %matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy import odr

# data
sd_x_real = [854881.6950868312, 2120597.826005219, 736273.1919717726, 2371889.81460371, 535179.270257414, 914730.5954746599]
sd_y_real = [0.0010542180764899318, 0.0002502402503244702, 0.0006222811989473171, 0.00021103416619461593, 0.00030252140616671673, 0.00031354722384367045]
x_real = [4924054.860560362, 15109167.31281077, 3832716.161527288, 12418192.213317377, 3160813.3126544063, 10049973.67477333]
y_real = [0.010512500258174207, 0.007139308311500112, 0.008902079935571126, 0.007830395195114538, 0.009383822481998878, 0.009074307023485157]

logx_real = np.log10(x_real)
logy_real = np.log10(y_real)
logsd_x_real = np.log10(sd_x_real)
logsd_y_real = np.log10(sd_y_real)

# def func(p, x):
#     a, b, c = p
#     return a * x *x + b*x + c

def func(p, x):
    a, b = p
    return a * x + b

# def func(p, x):
#     a, b = p
#     return a * x**b

# Model object
model = odr.Model(func)
n = 100
# beta0 = [0., 1., 1.]
beta0 = [0.1, -0.1]

# test data and error
# x0 = np.linspace(-10, 10, n)
x0 = np.linspace(np.min(logx_real), np.max(logx_real), n)
# y0 = - 0.07 * x0 * x0 + 0.5 * x0 + 2.
y0 = func([0.15, -0.15], x0)                 
# noise_x = np.random.normal(0.0, 1.0, len(x0))
noise_x = np.random.uniform(-np.max(logsd_x_real), np.max(logsd_x_real), len(x0))
# noise_y = np.random.normal(0.0, 1.0, len(x0))
noise_y = np.random.uniform(-np.max(logsd_y_real), np.max(logsd_y_real), len(x0))

y = y0 + noise_y
x = x0 + noise_x





# Create a RealData object
data = odr.RealData(x, y, sx=noise_x, sy=noise_y)

# Set up ODR with the model and data.
odr = odr.ODR(data, model, beta0=beta0)

# Run the regression.
out = odr.run()

#print fit parameters and 1-sigma estimates
popt = out.beta
perr = out.sd_beta
print('fit parameter 1-sigma error')
print('———————————–')
for i in range(len(popt)):
    print(str(popt[i])+' +- '+str(perr[i]))
print('———————————–')    
out.pprint()

# prepare confidence level curves
nstd = 1. # to draw 5-sigma intervals
popt_up = popt + nstd * perr
popt_dw = popt - nstd * perr

x_fit = np.linspace(min(x), max(x), 100)
fit = func(popt, x_fit)
fit_up = func(popt_up, x_fit)
fit_dw= func(popt_dw, x_fit)

#plot
fig, ax = plt.subplots(1, figsize=(10,10))
ax.errorbar(x, y, yerr=noise_y, xerr=noise_x, ecolor='k', fmt='None', label='data')
plt.xlabel('x', fontsize=18)
plt.ylabel('y', fontsize=18)
plt.title('fit with error on both axis', fontsize=18)
plt.plot(x_fit, fit, 'r', lw=2, label='best fit curve')
plt.plot(x0, y0, 'k:', lw=2, label='True curve')
ax.fill_between(x_fit, fit_up, fit_dw, alpha=.25, label='1-sigma interval')
ax.legend(fontsize=18)
plt.show()

ValueError: scale < 0

In [1]:
import pandas as pd
import numpy as np

filepath = 'https://raw.githubusercontent.com/selva86/datasets/master/diamonds.csv'
df = pd.read_csv(filepath).iloc[:, [0,4,6]]
df.head()

,carat,depth,price
0,0.23,61.5,326
1,0.21,59.8,326
2,0.23,56.9,327
3,0.29,62.4,334
4,0.31,63.3,335


In [8]:
def mahalanobis(x=None, data=None, cov=None):
    """Compute the Mahalanobis Distance between each row of x and the data  
    x    : vector or matrix of data with, say, p columns.
    data : ndarray of the distribution from which Mahalanobis distance of each observation of x is to be computed.
    cov  : covariance matrix (p x p) of the distribution. If None, will be computed from data.
    """
    x_minus_mu = x - np.mean(data)
    print('mean data\n', np.mean(data))
    print('x minus mu\n', x_minus_mu)
    if not cov:
        cov = np.cov(data.values.T)
    print('cov\n', cov)
    inv_covmat = np.linalg.inv(cov)
    left_term = np.dot(x_minus_mu, inv_covmat)
    mahal = np.dot(left_term, x_minus_mu.T)
    return mahal.diagonal()

df_x = df[['carat', 'depth', 'price']].head(500)
# print('df_x\n', df_x)
# print('df\n',df[['carat', 'depth', 'price']])
df_x['mahala'] = mahalanobis(x=df_x, data=df[['carat', 'depth', 'price']])
df_x.head()

mean data
 carat       0.797940
depth      61.749405
price    3932.799722
dtype: float64
x minus mu
        carat     depth        price
0   -0.56794 -0.249405 -3606.799722
1   -0.58794 -1.949405 -3606.799722
2   -0.56794 -4.849405 -3605.799722
3   -0.50794  0.650595 -3598.799722
4   -0.48794  1.550595 -3597.799722
..       ...       ...          ...
495 -0.08794  0.950595 -1110.799722
496 -0.08794 -0.449405 -1110.799722
497 -0.09794 -1.549405 -1110.799722
498 -0.09794 -1.049405 -1110.799722
499  0.10206  2.250595 -1110.799722

[500 rows x 3 columns]
cov
 [[ 2.24686660e-01  1.91665282e-02  1.74276536e+03]
 [ 1.91665282e-02  2.05240384e+00 -6.08537121e+01]
 [ 1.74276536e+03 -6.08537121e+01  1.59156294e+07]]


,carat,depth,price,mahala
0,0.23,61.5,326,1.709860
1,0.21,59.8,326,3.540097
2,0.23,56.9,327,12.715021
3,0.29,62.4,334,1.454469
4,0.31,63.3,335,2.347239
